In [1]:
#| default_exp advanced_wrapping

In [2]:
#| export

from blender_tissue_cartography import io as tcio
from blender_tissue_cartography import registration as tcreg
from blender_tissue_cartography import wrapping as tcwrap

import numpy as np
from copy import deepcopy
import warnings
import igl

from scipy import sparse
import shutil

In [3]:
from tqdm.notebook import tqdm
from importlib import reload

import matplotlib.pyplot as plt

In [4]:
np.set_printoptions(suppress=True)

## Advanced wrapping

In this notebook we present some additional wrapping algorithms. So far, we have wrapped two meshes by simply projection each source mesh vertex onto the closest position on the target mesh. This works, but has drawbacks when there are large, localized deformations.

We can think of the wrapping process like stretching the source surface to fit onto the target surface. The "closest point" method effectively gives the source surface zero elastic resistance during the stretching. This means that small regions will capture all deformations, leading to poor results.

Here, we'll look into alternative algorithms which implement some form of effective elasticity.

#### Loading test data

Let's load the test meshes from the fly midgut

In [5]:
meshes_dict = {i: tcio.read_other_formats_without_uv(f"movie_example/meshes/mesh_{str(i).zfill(2)}.ply")
               for i in tqdm(range(1, 32))}

  0%|          | 0/31 [00:00<?, ?it/s]

In [6]:
mesh_initial_UV = tcio.ObjMesh.read_obj("movie_example/initial_uv.obj")
mesh_final_UV = tcio.ObjMesh.read_obj("movie_example/final_uv.obj") # this is a UV map defined for tpt 20

### Reverse wrapping

In [13]:
reload(tcwrap)

<module 'blender_tissue_cartography.wrapping' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/wrapping.py'>

In [14]:
t_final = 20
meshes_wrapped_reverse = {20: mesh_final_UV}

In [15]:
for i in tqdm(reversed(range(1, t_final))):
    print(i)
    mesh_source = meshes_wrapped_reverse[i+1]
    mesh_target = meshes_dict[i]
    # register using ICP
    trafo_centroid_intertia, _ = tcreg.align_by_centroid_and_intertia(mesh_source.vertices, mesh_target.vertices,
                                    q=0.01, shear=True, choose_minimal_rotation=True)
    trafo_icp, _, _ = tcreg.icp(mesh_source.vertices, mesh_target.vertices, initial=trafo_centroid_intertia,
                                threshold=1e-4, max_iterations=20, include_scaling=True, n_samples=5000)
    mesh_registered = mesh_source.apply_affine_to_mesh(trafo_icp)
    # shrink-wrap
    mesh_wrapped = tcwrap.shrinkwrap_igl(mesh_registered, mesh_target,
                                         n_iter_smooth_target=2, n_iter_smooth_wrapped=2)
    mesh_wrapped = tcwrap.smooth_laplacian_on_surface(mesh_wrapped, n_iter=5, lamb=0.5, n_iter_laplace=10)
    # append to list
    mesh_wrapped.write_obj(f"movie_example/meshes_wrapped_reverse/mesh_{str(i).zfill(2)}_wrapped_reverse.obj")
    meshes_wrapped_reverse[i] = mesh_wrapped

0it [00:00, ?it/s]

19
18
17
16


/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/wrapping.py:192: RuntimeWarning: Warning: 2 normal(s) flipped during shrink-wrapping
  return mesh_wrapped


15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


### Non-rigid ICP

Try it out first using trimesh, then do myself to avoid dependency

In [17]:
from blender_tissue_cartography import interface_trimesh as inttri
import trimesh

### Thin-plate spline registration